In [14]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [15]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/pfannes/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [16]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


An NYC-based robotic company and kitchen solutions provider, Aniai, recently secured $12M in Pre-Series A funding, with InterVest leading the round and support from SV Investment, Ignite Innovation, and Capstone Partners. The company plans to use this investment to establish its first manufacturing facility, Factory One, in South Korea. This move is aimed at ensuring stable production and efficient supply chain management for their flagship product, the Alpha Grill. Aniai, founded in 2020 by CEO Gunpil Hwang, specializes in developing robotic solutions for cooking hamburgers. The Alpha Grill, their key product, utilizes robotics to cook hamburgers efficiently, with a double-sided grill capable of cooking up to eight patties simultaneously or 200 patties per hour. By automating tasks such as pressing and flipping patties, the Alpha Grill helps restaurant brands streamline operations, reduce cooking time, and decrease labor intensity for employees.

Gunpil Hwang, the CEO of Aniai, expres

In [17]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [18]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

#System prompt (for all users)
system_prompt_mom_test_problem = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the TRIZ method: 
        
        The TRIZ method is a toolbox of methods, i.e. a collection of different working methods for the successful creative processing of problems.
        TRIZ is particularly useful for analyzing and solving technical problems. Originating from mechanical engineering, this is where it shows its strength, but the tools can also be used to tackle electro-mechanical, electronic, process engineering, production engineering and even software architecture problems. 
        Modern TRIZ is understood to be classic TRIZ extended by further procedures. It is either new practices created for TRIZ or adaptations of other analytical and creative methods. Because the international TRIZ Level 5 certificate (equivalent to the TRIZ Master) is only awarded by a committee of 12 TRIZ Masters if the TRIZ candidate (prerequisite: MATRIZ Level 4, i.e. TRIZ participant) introduces a new method into TRIZ, the toolbox of methods expands with each new TRIZ Master.
        Of course, as with doctoral theses, not all of it makes it into the TRIZ community. Only the best are adopted and thus become an integral part of the method toolkit, which is then referred to as "Modern TRIZ".
   
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
mom_test_problems = []

user_prompt_mom_test_problem = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Please identify problems that are experienced by customers in the above text and use the following TRIZ methods from the perspective of one of these customers:

     - Functional analysis for products and processes with its extensions (incremental improvement, radical improvement, value analysis, patent avoidance, function theft)
        - Root cause analysis
        - Trimming
        - Effect databases
        - Feature transfer
        - Flow analysis
        - Advanced trends in technology evolution

    Please provide some sources of inspiration that you used from the document excerpts


    
    """
    
completion_mom_test = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_mom_test_problem},
            {"role": "user", "content": user_prompt_mom_test_problem}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
mom_test_problems.append(completion_mom_test.choices[0].message.content)
print(completion_mom_test.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


# Next agent takes mom-test output and comes up with solutions based on this
mom_test_solutions = []

system_prompt_mom_test_solution = f"""
    You are a highly qualified CTO with decades of experience in coming up with novel, exciting business opportunities.
"""

user_prompt_mom_test_solution = f"""

    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {mom_test_problems}
    ```    
  
    Please develop 5 really specific ideas/visions on how the problems outlined in the are delimited by ''' can be solved. The solutions should consider the following points:
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    For the solution, please write only 40 words. It should contain the following contents:
    1) A title for the solution in max 3 words
    2) Your solution summarized in 1 sentence
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) How the solutions is novel, that is, not already existing in the market
    5) Please provide some sources of inspiration that you used from the document excerpts.

    """
    

completion_mom_test_solution = openai.chat.completions.create(
model="gpt-3.5-turbo-0125",
messages=[
{"role": "system", "content": system_prompt_mom_test_solution},
{"role": "user", "content": user_prompt_mom_test_solution}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion_mom_test_solution.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
mom_test_solutions.append(completion_mom_test_solution.choices[0].message.content)


#Summary

summary_solutions = []

user_prompt_summarizer = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas:
    {mom_test_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (5 words max)
    “Solved Problem”: 1 sentence summary of the problem that is being solved and how much budget is needed
    “Solution”: Provide a short summary of the solution and how it works (4 to 5 sentences max)
    “Impact”: How does the solution solve the specific problem (3 sentences max)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (4 to 5 sentences max)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion_summarizer = openai.chat.completions.create(
model="gpt-3.5-turbo-0125",
messages=[
{"role": "system", "content": system_prompt_mom_test_solution},
{"role": "user", "content": user_prompt_summarizer}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion_summarizer.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion_summarizer.choices[0].message.content)



Based on the provided text on different companies in the field of factory automation, we can identify some key problems experienced by customers in the industry. Let's analyze these problems using TRIZ methods:

1. **Problem**: Lack of seamless integration and communication between different factory automation systems and equipment.
   
   **TRIZ Methods**:
   - **Root Cause Analysis**: Identify the main reasons causing the lack of integration, such as incompatible communication protocols, proprietary systems, or legacy equipment.
   
   - **Feature Transfer**: Look into successful integration solutions from other industries or companies within the document excerpts that have effectively integrated different systems. For example, Software Defined Automation's approach to cloud-based management and virtualization of PLCs could provide insights for feature transfer to address integration issues.
   
   - **Flow Analysis**: Analyze the flow of data and communication between different syst

In [19]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

First, repeat the following points for the first solution as found in the following text: {summary_solutions}

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0125",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating.append(completion_summarizer.choices[0].message.content)





**Data Fusion Hub**:
- **Name**: Data Fusion Hub
- **Solved Problem**: Streamlining factory automation operations with actionable insights
- **Solution**: Centralized platform integrating diverse automation systems for real-time data optimization
- **Impact**: Enhancing communication, collaboration, and efficiency while reducing operational costs
- **Technology**: Utilizes advanced data integration algorithms and real-time analytics for seamless system coordination
- **Sources**: Inspired by Cellares' Cell Shuttle technology for manufacturing process consolidation

**Functionality and Performance**: 3 - The solution effectively addresses the problem by streamlining operations and providing actionable insights.
**Scalability**: 3 - The centralized platform can adapt to future growth and changes in requirements.
**Cost-effectiveness**: 2 - The high upfront cost might hinder some organizations, but the return on investment is expected to be significant.
**Innovativeness**: 2 - While centr

In [20]:
user_prompt_3 = f""""

Please take all the solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest absolute rating out of 15 (please be very diligent in this task. If there is an overall rating of 13, it should for example be outputed instead of an idea with a score of 10 out of 15). Sort the three in descending rating order.

Repeat the followinng points for each of the top three:

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”
    - also include their rating from: {initial_rating} to double check they are in the top 3 highest rated


"""


completion = openai.chat.completions.create(
model="gpt-3.5-turbo-0125",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

Based on the ratings provided, the top three solutions for the problem of lack of seamless integration and communication between different factory automation systems and equipment are as follows:

1. **Data Fusion Hub**:
   - **Rating**: 14 out of 15
   - **Solved Problem**: Streamlining factory automation operations with actionable insights (Budget: High).
   - **Solution**: Centralized platform integrating diverse automation systems for real-time data optimization.
   - **Impact**: Enhancing communication, collaboration, and efficiency while reducing operational costs.
   - **Technology**: Utilizes advanced data integration algorithms and real-time analytics for seamless system coordination.
   - **Sources**: Inspired by Cellares' Cell Shuttle technology for manufacturing process consolidation.

2. **IoT Harmony Platform**:
   - **Rating**: 13 out of 15
   - **Solved Problem**: Unifying factory automation devices for predictive maintenance (Budget: High).
   - **Solution**: IoT ecosy